In [1]:
import pandas as pd
!gsutil cp gs://chem_dsrc/P0_Toxcast/Data/assay_df/*_df.csv ./

Copying gs://chem_dsrc/P0_Toxcast/Data/assay_df/335_all_assay_df.csv...
Copying gs://chem_dsrc/P0_Toxcast/Data/assay_df/NCCT_TPO_AUR_dn_df.csv...
Copying gs://chem_dsrc/P0_Toxcast/Data/assay_df/NVS_ADME_hCYP1A2_df.csv...
Copying gs://chem_dsrc/P0_Toxcast/Data/assay_df/NVS_ADME_hCYP2B6_df.csv...
/ [4 files][  4.3 MiB/  4.3 MiB]                                                
Operation completed over 4 objects/4.3 MiB.                                      


In [2]:
# rdkit 2020.03.3 버전 다운로드
!pip install kora -q
import kora.install.rdkit
# !pip install pycaret
# from pycaret.classification import *
import numpy
import pandas
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys

from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import matthews_corrcoef

from imblearn.combine import SMOTEENN 
from sklearn import metrics

     |████████████████████████████████| 61kB 3.6MB/s 
     |████████████████████████████████| 61kB 7.8MB/s 


/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [3]:
import os 
os.mkdir("./result_sklearn")
os.mkdir("./result_sklearn/dup_smiles/")
os.mkdir("./result_sklearn/err_smiles/")
os.mkdir("./result_sklearn/metric/")



In [23]:
def g_tf(ss) : 
  # ss = selected_assay_list[assay_index] # 어세이 이름ss = selected_assay_list[assay_index] # 어세이 이름
  a_i = 0 # assay index
  ffpp = "maccs"

  df_ss = df.loc[:,['smiles',ss]]
  df_ss = df_ss.dropna()
  assays_ss = df_ss.columns

  #Smiles --> MACCS Keys
  assay = ss
  df_train  = df_ss[['smiles',ss] ]

  smiles_list = []
  err_smiles = []# fingerfrint로 변환시 에러 smiles
  fps = []
  enc_y = []
  for index, row in df_train.iterrows():

    if row[-1] == 'N' : 
      target = 0
    else :
      target = 1

    try : 
      mol = Chem.MolFromSmiles(row['smiles'])
      
      if ffpp == 'maccs' :    
          fp = MACCSkeys.GenMACCSKeys(mol)
      elif ffpp == 'morgan' : 
          fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
      elif ffpp == 'rdkit' : 
          fp = Chem.RDKFingerprint(mol)
      elif ffpp == 'pattern' : 
          fp = Chem.rdmolops.PatternFingerprint(mol)
      elif ffpp == 'layerd' : 
          fp = Chem.rdmolops.LayeredFingerprint(mol)
      else : 
          print("fingerfrint selection error!")
          
      smiles_list.append(row['smiles'])   
      fps.append(fp)
      enc_y.append(target)
    except : 
      err_smiles.append(row['smiles'])
      pass

  np_fps = []
  for fp in fps:
    arr = numpy.zeros((1,))
    DataStructs.ConvertToNumpyArray(fp, arr)
    np_fps.append(arr)

  np_fps_array = numpy.array(np_fps)

  print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

  x_train_res, y_train_res = np_fps_array, np.array(enc_y)
  train = pd.DataFrame(data=x_train_res)
  train['label'] = y_train_res

  d = pd.DataFrame(columns = ['smiles','Type'] )
  d['smiles'] = smiles_list
  d['Type'] = enc_y

  #모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
  mychems = []
  for item in np_fps:
      #print(item)
      item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
      s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
      t = [str(i) for i in s]
      mychems.append(''.join(t))

  i = 0
  counter = 0
  chemdict = {} #딗셔너리.
  listofdups = []
  for item in mychems:
      try:
          chemdict[str(item)].append(i)
          #print('dup found')
          for j in range(len(chemdict[str(item)])):
              if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                  listofdups.append(i)
                  counter+=1
      except: 
          #print('adding new')
          chemdict[str(item)] = []
          chemdict[str(item)].append(i)
      i+=1
  print(str(counter)+' '+str(len(list(set(listofdups)))))
  #smote적용하기 전에서 listofdups를 빼면 된다!

  dups_list = list(set(listofdups))

  dups_list_0 = []
  for index in dups_list : 
    if train.iloc[index].label == 0 :
      dups_list_0.append(index)

  train = train.drop(dups_list_0)

  duplicated_smiles_0_df = pd.DataFrame()
  duplicated_smiles_0_df['deleted_duplicate_smiles_0'] = smiles_list
  duplicated_smiles_0_df = duplicated_smiles_0_df.iloc[dups_list_0,]
  duplicated_smiles_0_df

  save_path = 'gs://chem_dsrc/P0_Toxcast/result_sklearn/'

  dupdf = "./result_sklearn/dup_smiles/"+ ss+"_dup_smiles_df.csv"
  duplicated_smiles_0_df.to_csv(dupdf)
  duppath = save_path + 'dup_smiles/'
  # !gsutil cp $dupdf $duppath

  err_smiles_df = pd.DataFrame(columns = [ss], index = ['err_n','err_smiles'])
  err_smiles_df[ss] = [ len(err_smiles), err_smiles]

  ersdf = "./result_sklearn/err_smiles/"+ ss+"_err_smiles_df.csv"
  err_smiles_df.to_csv(ersdf)
  errpath = save_path + 'err_smiles/'
  # !gsutil cp $ersdf $errpath $errpath

  rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=36851234) 
  sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

  X = np.array( train.drop('label', axis =1) )
  y = np.array(train.label)

  count = 0 
  rep = 0

  metric_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])
  metric_fold_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])

  for train_index, test_index in rskf.split(X, y): 
    # print(count)
    # print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    if ss in ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn'] : 
      pass
    else : 
      X_train, y_train = sme.fit_sample(X_train, list(y_train) )
    
    clf = RandomForestClassifier()
    clf.fit(X_train, y_train)

    result =  clf.predict_proba(X_test) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
    y_pred = clf.predict(X_test)
    # print(result)
    # print('y_pred : ', y_pred)
    # print('y_test: ' ,y_test)
    conf_mat = confusion_matrix(y_test,y_pred)
    tn, fp, fn, tp = conf_mat.ravel()

    auc =  roc_auc_score(y_test,result[:,1] )
    recall = tp/(tp+fn) # 실제 1인 것 중 모델이 1이라 한것의 비율
    specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율
    b_a = (recall + specificity) / 2
    precision =  tp / (tp+fp)
    F1 = 2* (precision * recall) / (precision + recall)
    MCC = matthews_corrcoef(y_test,y_pred)
    accuracy = clf.score(X_test,y_test)

    # print( "AUC :  " , auc )
    # print( "B_A : ",b_a) # 민감도(recall) + 특이도 / 2
    # # print( "recall : ", recall_score(y_test, y_pred, average=None), recall_score(y_test, y_pred, average='binary'),  recall_score(y_test, y_pred), tp/(tp+fn) )
    # print( "recall : ", recall)
    # print( "precision : ", precision )
    # print("F1 : ", F1 )
    # print("MCC : ", MCC)
    # print( "accuracy : ",accuracy )

    metric_fold_df = metric_fold_df.append(pd.DataFrame([[auc,b_a,recall,precision,F1,MCC,accuracy]],columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy']) )
    # print()
    count +=1
    if count % 10 == 0 :
      average = metric_fold_df.mean()
      std = metric_fold_df.std()
      metric_fold_df = metric_fold_df.append(average.to_frame().T)
      metric_fold_df = metric_fold_df.append(std.to_frame().T)
      metric_fold_df = round(metric_fold_df ,4)
      metric_fold_df.index = [str(rep) +'_' +  str(i) for i in range(10)]+ [str(rep)+'_Mean' , str(rep)+'_SD']
      metric_df = metric_df.append(metric_fold_df)
      metric_fold_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])
      rep += 1
      print(ss, rep)

  #전체 평균내기.
  bool_list = [] 
  for i in metric_df.index :
    if'Mean' in i  or 'SD' in i : 
      bool_list.append(False) 
    else : 
      bool_list.append(True)

  average = metric_df[bool_list].mean().to_frame().T
  average.index = ['Mean']
  std = metric_df[bool_list].std().to_frame().T
  std.index =['SD']

  metric_df = metric_df.append(round(average,4) )
  metric_df = metric_df.append(round(std,4))
  metric_df.to_csv("./result_sklearn/metric/"+ss+"_metric.csv")
  metric_file_path = ss+"_metric.csv"
  result_path = save_path + 'metric/'
  # !gsutil cp $metric_file_path  $result_path
    

In [24]:

selected_assay_list = ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn','TOX21_AhR_LUC_Agonist', 'TOX21_Aromatase_Inhibition', 'TOX21_VDR_BLA_agonist_ratio', 'TOX21_VDR_BLA_antagonist_ratio', 'TOX21_AR_BLA_Agonist_ratio', 'TOX21_AR_BLA_Antagonist_ratio', 'TOX21_AR_LUC_MDAKB2_Agonist', 'TOX21_p53_BLA_p1_ratio', 'TOX21_p53_BLA_p2_ratio', 'TOX21_p53_BLA_p3_ratio', 'TOX21_p53_BLA_p4_ratio', 'TOX21_p53_BLA_p5_ratio', 'TOX21_HSE_BLA_agonist_ratio', 'TOX21_ELG1_LUC_Agonist', 'TOX21_ERa_BLA_Agonist_ratio', 'TOX21_ERa_BLA_Antagonist_ratio', 'TOX21_ESRE_BLA_ratio', 'TOX21_FXR_BLA_agonist_ratio', 'TOX21_FXR_BLA_antagonist_ratio', 'TOX21_GR_BLA_Agonist_ratio', 'TOX21_GR_BLA_Antagonist_ratio', 'TOX21_NFkB_BLA_agonist_ratio', 'TOX21_ARE_BLA_agonist_ratio', 'TOX21_TR_LUC_GH3_Agonist', 'TOX21_TR_LUC_GH3_Antagonist', 'TOX21_PPARg_BLA_Agonist_ratio', 'TOX21_PPARd_BLA_agonist_ratio', 'TOX21_PPARd_BLA_antagonist_ratio', 'TOX21_PPARg_BLA_antagonist_ratio']
selected_assay_list.remove('TOX21_NFkB_BLA_agonist_ratio')

assay_name = '335_all_assay'
df = pd.read_csv(assay_name+"_df.csv")

In [25]:
len(selected_assay_list)

31

In [26]:
import warnings
warnings.filterwarnings(action='ignore')

# def fxn():
#     warnings.warn("deprecated", DeprecationWarning)

# with warnings.catch_warnings():
#     warnings.simplefilter("ignore")
#     fxn()

In [27]:
import multiprocessing
CPU_CORE = multiprocessing.cpu_count() # 멀티프로세싱 CPU 사용 수
if __name__=='__main__':
    pool = multiprocessing.Pool(processes=CPU_CORE)
    pool.map(g_tf, selected_assay_list)
    pool.close()
    pool.join()  

변환 에러난 smiles 개수 :  0 []
0 0
변환 에러난 smiles 개수 :  0 []
1 1
변환 에러난 smiles 개수 :  2 ['CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1']
27 18
NVS_ADME_hCYP2B6 1
NVS_ADME_hCYP1A2 1
NCCT_TPO_AUR_dn 1
NVS_ADME_hCYP2B6 2
NVS_ADME_hCYP1A2 2
변환 에러난 smiles 개수 :  7 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
변환 에러난 smiles 개수 :  7 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[

In [28]:
os.listdir('/content/result_sklearn/metric/')

['TOX21_GR_BLA_Antagonist_ratio_metric.csv',
 'TOX21_ARE_BLA_agonist_ratio_metric.csv',
 'TOX21_ERa_BLA_Agonist_ratio_metric.csv',
 'TOX21_PPARg_BLA_Agonist_ratio_metric.csv',
 'TOX21_ELG1_LUC_Agonist_metric.csv',
 'TOX21_AR_BLA_Agonist_ratio_metric.csv',
 'TOX21_PPARg_BLA_antagonist_ratio_metric.csv',
 'TOX21_GR_BLA_Agonist_ratio_metric.csv',
 'NCCT_TPO_AUR_dn_metric.csv',
 'TOX21_p53_BLA_p5_ratio_metric.csv',
 'TOX21_PPARd_BLA_antagonist_ratio_metric.csv',
 'TOX21_p53_BLA_p2_ratio_metric.csv',
 'TOX21_HSE_BLA_agonist_ratio_metric.csv',
 'TOX21_p53_BLA_p1_ratio_metric.csv',
 'TOX21_AR_LUC_MDAKB2_Agonist_metric.csv',
 'NVS_ADME_hCYP1A2_metric.csv',
 'TOX21_VDR_BLA_agonist_ratio_metric.csv',
 'TOX21_AR_BLA_Antagonist_ratio_metric.csv',
 'TOX21_TR_LUC_GH3_Agonist_metric.csv',
 'NVS_ADME_hCYP2B6_metric.csv',
 'TOX21_AhR_LUC_Agonist_metric.csv',
 'TOX21_TR_LUC_GH3_Antagonist_metric.csv',
 'TOX21_VDR_BLA_antagonist_ratio_metric.csv',
 'TOX21_ESRE_BLA_ratio_metric.csv',
 'TOX21_FXR_BLA_agoni

In [29]:
df_list = [] 
for i in os.listdir('/content/result_sklearn/metric/') :
  df_list.append(pd.read_csv('/content/result_sklearn/metric/'+i) )

In [30]:
df_list[8]

,Unnamed: 0,AUC,B_A,Recall,Precision,F1,MCC,Accuracy
0,0_0,0.7090,0.6146,0.5417,0.5652,0.5532,0.2305,0.6250
1,0_1,0.6826,0.5742,0.6000,0.5172,0.5556,0.1476,0.5714
2,0_2,0.7723,0.7271,0.6800,0.7083,0.6939,0.4563,0.7321
3,0_3,0.8026,0.6432,0.4800,0.6667,0.5581,0.3049,0.6607
4,0_4,0.8581,0.7594,0.6800,0.7727,0.7234,0.5280,0.7679
...,...,...,...,...,...,...,...,...
117,9_9,0.7292,0.7043,0.6667,0.6667,0.6667,0.4086,0.7091
118,9_Mean,0.7705,0.7052,0.6552,0.6875,0.6665,0.4165,0.7114
119,9_SD,0.0517,0.0392,0.0782,0.0626,0.0464,0.0793,0.0402
120,Mean,0.7778,0.7063,0.6528,0.6879,0.6665,0.4180,0.7124


In [32]:
!gsutil cp -r ./result_sklearn gs://chem_dsrc/P0_Toxcast/

Copying file://./result_sklearn/dup_smiles/TOX21_p53_BLA_p1_ratio_dup_smiles_df.csv [Content-Type=text/csv]...
Copying file://./result_sklearn/dup_smiles/TOX21_AhR_LUC_Agonist_dup_smiles_df.csv [Content-Type=text/csv]...
Copying file://./result_sklearn/dup_smiles/TOX21_HSE_BLA_agonist_ratio_dup_smiles_df.csv [Content-Type=text/csv]...
Copying file://./result_sklearn/dup_smiles/TOX21_ELG1_LUC_Agonist_dup_smiles_df.csv [Content-Type=text/csv]...
/ [4 files][  3.3 KiB/  3.3 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./result_sklearn/dup_smiles/TOX21_AR_BLA_Antagonist_ratio_dup_smiles_df.csv [Content-Type=text/csv]...
Copying file://./result_sklearn/dup_smiles/TOX21_PPARd_BLA_agonist_ratio_dup_smiles_df.csv [Content-T

In [17]:
# def g_tf(ss) : 
assay_index = 4
ss = selected_assay_list[assay_index] # 어세이 이름ss = selected_assay_list[assay_index] # 어세이 이름
a_i = 0 # assay index
ffpp = "maccs"

df_ss = df.loc[:,['smiles',ss]]
df_ss = df_ss.dropna()
assays_ss = df_ss.columns

#Smiles --> MACCS Keys
assay = ss
df_train  = df_ss[['smiles',ss] ]

smiles_list = []
err_smiles = []# fingerfrint로 변환시 에러 smiles
fps = []
enc_y = []
for index, row in df_train.iterrows():

  if row[-1] == 'N' : 
    target = 0
  else :
    target = 1

  try : 
    mol = Chem.MolFromSmiles(row['smiles'])
    
    if ffpp == 'maccs' :    
        fp = MACCSkeys.GenMACCSKeys(mol)
    elif ffpp == 'morgan' : 
        fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    elif ffpp == 'rdkit' : 
        fp = Chem.RDKFingerprint(mol)
    elif ffpp == 'pattern' : 
        fp = Chem.rdmolops.PatternFingerprint(mol)
    elif ffpp == 'layerd' : 
        fp = Chem.rdmolops.LayeredFingerprint(mol)
    else : 
        print("fingerfrint selection error!")
        
    smiles_list.append(row['smiles'])   
    fps.append(fp)
    enc_y.append(target)
  except : 
    err_smiles.append(row['smiles'])
    pass

np_fps = []
for fp in fps:
  arr = numpy.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  np_fps.append(arr)

np_fps_array = numpy.array(np_fps)

print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

x_train_res, y_train_res = np_fps_array, np.array(enc_y)
train = pd.DataFrame(data=x_train_res)
train['label'] = y_train_res

d = pd.DataFrame(columns = ['smiles','Type'] )
d['smiles'] = smiles_list
d['Type'] = enc_y

#모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
mychems = []
for item in np_fps:
    #print(item)
    item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
    s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
    t = [str(i) for i in s]
    mychems.append(''.join(t))

i = 0
counter = 0
chemdict = {} #딗셔너리.
listofdups = []
for item in mychems:
    try:
        chemdict[str(item)].append(i)
        #print('dup found')
        for j in range(len(chemdict[str(item)])):
            if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                listofdups.append(i)
                counter+=1
    except: 
        #print('adding new')
        chemdict[str(item)] = []
        chemdict[str(item)].append(i)
    i+=1
print(str(counter)+' '+str(len(list(set(listofdups)))))
#smote적용하기 전에서 listofdups를 빼면 된다!

dups_list = list(set(listofdups))

dups_list_0 = []
for index in dups_list : 
  if train.iloc[index].label == 0 :
    dups_list_0.append(index)

train = train.drop(dups_list_0)

duplicated_smiles_0_df = pd.DataFrame()
duplicated_smiles_0_df['deleted_duplicate_smiles_0'] = smiles_list
duplicated_smiles_0_df = duplicated_smiles_0_df.iloc[dups_list_0,]
duplicated_smiles_0_df

save_path = 'gs://chem_dsrc/P0_Toxcast/result_sklearn/'

dupdf = "./result_sklearn/dup_smiles/"+ ss+"_dup_smiles_df.csv"
duplicated_smiles_0_df.to_csv(dupdf)
duppath = save_path + 'dup_smiles/'
# !gsutil cp $dupdf $duppath

err_smiles_df = pd.DataFrame(columns = [ss], index = ['err_n','err_smiles'])
err_smiles_df[ss] = [ len(err_smiles), err_smiles]

ersdf = "./result_sklearn/err_smiles/"+ ss+"_err_smiles_df.csv"
err_smiles_df.to_csv(ersdf)
errpath = save_path + 'err_smiles/'
# !gsutil cp $ersdf $errpath $errpath

rskf = RepeatedStratifiedKFold(n_splits=10, n_repeats=10, random_state=36851234) 
sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

X = np.array( train.drop('label', axis =1) )
y = np.array(train.label)

count = 0 
rep = 0

metric_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])
metric_fold_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])

for train_index, test_index in rskf.split(X, y): 
  # print(count)
  # print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  if ss in  ['NVS_ADME_hCYP1A2' , 'NVS_ADME_hCYP2B6' , 'NCCT_TPO_AUR_dn']  : 
    pass
  else : 
    X_train, y_train = sme.fit_sample(X_train, y_train)
  
  clf = RandomForestClassifier()
  clf.fit(X_train, y_train)

  result =  clf.predict_proba(X_test) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
  y_pred = clf.predict(X_test)
  # print(result)
  # print('y_pred : ', y_pred)
  # print('y_test: ' ,y_test)
  conf_mat = confusion_matrix(y_test,y_pred)
  tn, fp, fn, tp = conf_mat.ravel()

  auc =  roc_auc_score(y_test,result[:,1] )
  recall = tp/(tp+fn) # 실제 1인 것 중 모델이 1이라 한것의 비율
  specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율
  b_a = (recall + specificity) / 2
  precision =  tp / (tp+fp)
  F1 = 2* (precision * recall) / (precision + recall)
  MCC = matthews_corrcoef(y_test,y_pred)
  accuracy = clf.score(X_test,y_test)

  # print( "AUC :  " , auc )
  # print( "B_A : ",b_a) # 민감도(recall) + 특이도 / 2
  # # print( "recall : ", recall_score(y_test, y_pred, average=None), recall_score(y_test, y_pred, average='binary'),  recall_score(y_test, y_pred), tp/(tp+fn) )
  # print( "recall : ", recall)
  # print( "precision : ", precision )
  # print("F1 : ", F1 )
  # print("MCC : ", MCC)
  # print( "accuracy : ",accuracy )

  metric_fold_df = metric_fold_df.append(pd.DataFrame([[auc,b_a,recall,precision,F1,MCC,accuracy]],columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy']) )
  # print()
  count +=1
  if count % 10 == 0 :
    average = metric_fold_df.mean()
    std = metric_fold_df.std()
    metric_fold_df = metric_fold_df.append(average.to_frame().T)
    metric_fold_df = metric_fold_df.append(std.to_frame().T)
    metric_fold_df = round(metric_fold_df ,4)
    metric_fold_df.index = [str(rep) +'_' +  str(i) for i in range(10)]+ [str(rep)+'_Mean' , str(rep)+'_SD']
    metric_df = metric_df.append(metric_fold_df)
    metric_fold_df = pd.DataFrame(columns = ['AUC','B_A','Recall','Precision','F1','MCC','Accuracy'])
    rep += 1
    print(ss, rep)

#전체 평균내기.
bool_list = [] 
for i in metric_df.index :
  if'Mean' in i  or 'SD' in i : 
    bool_list.append(False) 
  else : 
    bool_list.append(True)

average = metric_df[bool_list].mean().to_frame().T
average.index = ['Mean']
std = metric_df[bool_list].std().to_frame().T
std.index =['SD']

metric_df = metric_df.append(round(average,4) )
metric_df = metric_df.append(round(std,4))
metric_df.to_csv("./result_sklearn/metric/"+ss+"_metric.csv")
metric_file_path = ss+"_metric.csv"
result_path = save_path + 'metric/'
# !gsutil cp $metric_file_path  $result_path
  

변환 에러난 smiles 개수 :  9 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[H][O]1[Au+][S-]C2O[C@H](CO)[C@@H](O)[C@H](O)[C@@]12[H]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[NH4+].[NH4+].F[Si--](F)(F)(F)(F)F', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
51 34


Process ForkPoolWorker-147:
Process ForkPoolWorker-86:
Process ForkPoolWorker-108:
Process ForkPoolWorker-141:
Process ForkPoolWorker-107:
Process ForkPoolWorker-159:
Process ForkPoolWorker-139:
Process ForkPoolWorker-112:
Process ForkPoolWorker-143:
Process ForkPoolWorker-104:
Process ForkPoolWorker-94:
Process ForkPoolWorker-96:
Process ForkPoolWorker-152:
Process ForkPoolWorker-122:
Process ForkPoolWorker-126:
Process ForkPoolWorker-93:
Process ForkPoolWorker-90:
Process ForkPoolWorker-88:
Process ForkPoolWorker-101:
Process ForkPoolWorker-81:
Process ForkPoolWorker-92:
Process ForkPoolWorker-85:
Process ForkPoolWorker-130:
Process ForkPoolWorker-102:
Process ForkPoolWorker-103:
Process ForkPoolWorker-129:
Process ForkPoolWorker-105:
Process ForkPoolWorker-133:
Process ForkPoolWorker-109:
Process ForkPoolWorker-89:
Process ForkPoolWorker-140:
Process ForkPoolWorker-127:
Process ForkPoolWorker-100:
Process ForkPoolWorker-155:
Process ForkPoolWorker-142:
Process ForkPoolWorker-118:
Pr

KeyboardInterrupt: ignored

In [19]:
X_train.shape

(7305, 167)

In [20]:
y_train.shape

(7305,)

In [21]:
y_test.shape

(812,)

In [22]:
sme.fit_sample(X_train, y_train)

(array([[0.       , 0.       , 0.       , ..., 1.       , 1.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 1.       , 1.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 1.       , 1.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 0.9356763, 1.       ,
         0.0643237],
        [0.       , 0.       , 0.       , ..., 1.       , 1.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 1.       , 1.       ,
         0.       ]]), array([0, 0, 0, ..., 1, 1, 1]))

In [22]:
metric_df

,AUC,B_A,Recall,Precision,F1,MCC,Accuracy
0_0,0.4875,0.4625,0.8000,0.5333,0.6400,-0.1000,0.5000
0_1,0.5812,0.6250,1.0000,0.6250,0.7692,0.3953,0.6667
0_2,0.6500,0.6250,1.0000,0.6250,0.7692,0.3953,0.6667
0_3,0.6494,0.5325,0.6364,0.6364,0.6364,0.0649,0.5556
0_4,0.5844,0.4610,0.6364,0.5833,0.6087,-0.0806,0.5000
...,...,...,...,...,...,...,...
9_9,0.6429,0.5929,0.9000,0.6429,0.7500,0.2398,0.6471
9_Mean,0.6263,0.5940,0.7755,0.6463,0.7002,0.2232,0.6239
9_SD,0.1995,0.1177,0.1747,0.0984,0.1161,0.2584,0.1222
Mean,0.6243,0.5957,0.7689,0.6518,0.7008,0.2131,0.6240


In [17]:
average

,AUC,B_A,Recall,Precision,F1,MCC,Accuracy
Mean,0.614469,0.581895,0.752186,0.640749,0.687243,0.184477,0.610073


In [18]:
std

,AUC,B_A,Recall,Precision,F1,MCC,Accuracy
SD,0.122989,0.110455,0.153075,0.086128,0.103796,0.245409,0.111451


In [21]:
round(std,4)

,AUC,B_A,Recall,Precision,F1,MCC,Accuracy
SD,0.1297,0.1016,0.1391,0.0793,0.0904,0.2255,0.1006
